In [1]:
from datetime import datetime
from dotenv import load_dotenv
from os import getenv
import pandas as pd
import requests

load_dotenv()
pd.set_option('display.max_columns', None)

/tmp/ipykernel_122440/2894595129.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv('races.csv')
df.when = pd.to_datetime(df.when)
df.head(2)

,division,when,team1,team2,time_tot,win,hurdles,name1,start_chng1,time1,name2,start_chng2,time2,name3,start_chng3,time3,name4,start_chng4,time4,event,course_color
0,1,2023-04-22 10:11:57,WildRunners dreamers,WildRunners Maximini,17.01,L,25.0,Sunny,-0.24,4.21,Dixie,0.30,4.24,Alex,0.16,4.22,Hector,0.0,4.13,Hanácké škvarek (Heroltice),blue
1,1,2023-04-22 10:11:57,WildRunners Maximini,WildRunners dreamers,19.43,L,25.0,Brooke,-0.11,4.08,Rocky,1.29,5.15,Bella,0.23,4.52,Bree,0.0,4.28,Hanácké škvarek (Heroltice),red


In [3]:
coordinates = {}

def coordinates_of(city):
    if city in coordinates:
        return coordinates[city]['lat'], coordinates[city]['lon']
    
    country = 'HU' if city == 'Tápiószentmárton' else 'CZ'
    url = f"https://api.openweathermap.org/geo/1.0/direct?q={city},{country}&appid={getenv('API_KEY')}"
    response = requests.get(url)
    data = response.json()[0]
    coordinates[city] = {
        'lat': data['lat'],
        'lon': data['lon']
    }
    return data['lat'], data['lon']

In [4]:
def get_timestamp(date, hour, minute):
    return int(datetime.strptime(f'{date} {hour}:{minute}:00', '%Y-%m-%d %H:%M:%S').timestamp())

In [5]:
def get_weather(city, date, hour, minute):
    lat, lon = coordinates_of(city)
    timestamp = get_timestamp(date, hour, minute)
    url = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat={lat}&lon={lon}&dt={timestamp}&units=metric&appid={getenv('API_KEY')}"
    response = requests.get(url)
    data = response.json()['data'][0]
    return data



In [6]:
def date2city(date):
    return df[df.when.dt.date == date].iloc[0].event.split('(')[1].split(')')[0]

In [8]:
days = pd.Series(df.when.dt.date.unique())

weather_df = pd.DataFrame({
    'date': days,
    'city': days.apply(date2city)
})
weather_df = weather_df.loc[weather_df.index.repeat(11)].reset_index(drop=True)
weather_df['hour'] = list(range(8, 19)) * len(days)
weather_df = weather_df.loc[weather_df.index.repeat(6)].reset_index(drop=True)
weather_df['minute'] = [0, 10, 20, 30, 40, 50] * (len(weather_df) // 6)
weather_df['temperature'] = 0.0
weather_df['feels_like'] = 0.0
weather_df['humidity'] = 0
weather_df['wind_speed'] = 0.0
weather_df['weather'] = ''
weather_df['rain'] = 0.0
weather_df.head()

,date,city,hour,minute,temperature,feels_like,humidity,wind_speed,weather,rain
0,2023-04-22,Heroltice,8,0,0.0,0.0,0,0.0,,0.0
1,2023-04-22,Heroltice,8,10,0.0,0.0,0,0.0,,0.0
2,2023-04-22,Heroltice,8,20,0.0,0.0,0,0.0,,0.0
3,2023-04-22,Heroltice,8,30,0.0,0.0,0,0.0,,0.0
4,2023-04-22,Heroltice,8,40,0.0,0.0,0,0.0,,0.0


In [10]:
def fill_weather_data(df):
    for i, row in df.iterrows():
        print(f'{row.city} - {row.date} {row.hour}:{row.minute}')
        weather = get_weather(row.city, str(row.date), str(row.hour), str(row.minute))
        df.loc[i, 'temperature'] = weather['temp']
        df.loc[i, 'feels_like'] = weather['feels_like']
        df.loc[i, 'humidity'] = weather['humidity']
        df.loc[i, 'wind_speed'] = weather['wind_speed']
        df.loc[i, 'weather'] = weather['weather'][0]['main']
        df.loc[i, 'rain'] = weather['rain']['1h'] if 'rain' in weather else 0.0

In [13]:
fill_weather_data(weather_df[weather_df.temperature == 0.0])

Blšany - 2023-09-04 11:40


KeyError: 'data'

In [12]:
weather_df

,date,city,hour,minute,temperature,feels_like,humidity,wind_speed,weather,rain
0,2023-04-22,Heroltice,8,0,8.99,7.80,74,2.26,Clear,0.0
1,2023-04-22,Heroltice,8,10,9.55,8.46,71,2.26,Clear,0.0
2,2023-04-22,Heroltice,8,20,10.14,9.04,70,2.26,Clear,0.0
3,2023-04-22,Heroltice,8,30,10.70,9.63,69,2.26,Clear,0.0
4,2023-04-22,Heroltice,8,40,13.33,12.26,59,2.26,Clear,0.0
...,...,...,...,...,...,...,...,...,...,...
1447,2023-07-30,Tápiószentmárton,18,10,0.00,0.00,0,0.00,,0.0
1448,2023-07-30,Tápiószentmárton,18,20,0.00,0.00,0,0.00,,0.0
1449,2023-07-30,Tápiószentmárton,18,30,0.00,0.00,0,0.00,,0.0
1450,2023-07-30,Tápiószentmárton,18,40,0.00,0.00,0,0.00,,0.0


In [120]:
weather_df.to_csv('weather.csv', index=False)